In [4]:
import requests
import json
class ProdSysApi:
    
    def __init__(self, url_list, url_single, user, password):
        self._auth = requests.auth.HTTPBasicAuth(user, password)
        self._urls = [url_single, url_list]
        
    def api_url(self, id=None):
        if id is None:
            return self._urls[1]
        else:
            return self._urls[0].format(id)
    
    def list(self):
        response = requests.get(self.api_url(), auth=self._auth)
        print(len(response.content))
        data = response.json()
        return data
        
    def get(self, id):
        response = requests.get(self.api_url(id), auth=self._auth)
        data = response.json()
        return data
    
    def new(self, data):
            return requests.post(
                self.api_url(), 
                auth = self._auth,
                json = data,
            )
        
    def patch(self, id, data):
        if 'api/indesign/' in self._urls[0]:
            return requests.post(
                self.api_url(id), 
                auth = self._auth,
                data = {'json': json.dumps(data)},
            )
        else:
            return requests.patch(
                self.api_url(id), 
                auth = self._auth,
                json = data,
            )
            

        
old_api = ProdSysApi(
    user='api',
    password='Avier1gi',
    url_list = 'http://universitas.no/admin/api/indesign/json/',
    url_single = 'http://universitas.no/admin/api/indesign/{}/json/',
)
new_api = ProdSysApi(
    user='api',
    password='Avier1gi',
    url_list = 'http://universitas.no/api/legacy/?limit=500',
    url_single = 'http://universitas.no/api/legacy/{}/',
)
local_api = ProdSysApi(
    user='api',
    password='api',
    url_list = 'http://web:8000/api/legacy/?limit=10',
    url_single = 'http://web:8000/api/legacy/{}/',
)



In [5]:
prodsaker = [old_api.get(ps['prodsak_id']) for ps in old_api.list() if ps['produsert'] < '8']


407940


In [6]:
len(prodsaker)

81

In [8]:
# Remove work in progress stories
qs = Story.objects.filter(publication_status__lt=Story.STATUS_FROM_DESK)
qs.delete()


def pre_save(model_instance, add):
    """ Returns field's value just before saving. """
    return getattr(model_instance, 'modified')

def update_timestamps(story, created, modified):
    story.created = created 
    story.modified = modified
    story._meta.get_field('modified').pre_save = pre_save
    story.save()


import pytz
def get_status(produsert, flagg, **kwargs):
    flag_status = {
        1: Story.STATUS_JOURNALIST,
        2: Story.STATUS_SUBEDITOR,
        3: Story.STATUS_EDITOR,
        4: Story.STATUS_EDITOR
    }.get(int(flagg))
    status = {
        Prodsak.DRAFT: flag_status,
        Prodsak.OLD_DRAFT: flag_status,
        Prodsak.READY_FOR_PRINT: Story.STATUS_TO_DESK,
        Prodsak.IMPORTED_TO_INDESIGN: Story.STATUS_AT_DESK,
        Prodsak.EXPORTED_FROM_INDESIGN: Story.STATUS_FROM_DESK,
    }.get(int(produsert))
    return status


def timeparser(value):
    dt=pytz.datetime.datetime.strptime(value, '%Y-%m-%d %H:%M:%S')
    return pytz.timezone('Europe/Oslo').localize(dt)
# for prodsak in filter(lambda s: len(s['mappe']) == 2, prodsaker):

for ps in prodsaker:
    ps['produsert'] = get_status(**ps)
    if ps['mappe'] == '\ufffdr':
        ps['mappe'] = 'år'
    created = timeparser(ps.get('dato'))
    modified = timeparser(ps.get('version_date'))
    #continue
    response = local_api.new(ps)
    print('{prodsak_id} {mappe} {arbeidstittel} {response.status_code}'.format(
        **ps, response=response))
    if response.status_code == 201:
        data = response.json()
        story = Story.objects.get(pk=data['prodsak_id'])
        update_timestamps(story, created, modified)
        print(modified - created)
        print(story, story.created-created, story.modified-modified)
    elif response.status_code == 400:
        ...
    else:
        print(response.status_code)
        print(json.dumps(ps, ensure_ascii=False, indent=2))
        error = response.text.split('Traceback (most recent call last):')[-1].strip()
        print(error.splitlines()[-1])
        print(error)
        qs = Story.objects.filter(publication_status__lt=Story.STATUS_FROM_DESK)
        qs.delete()
        break

23260 vispor 17 Vi spør 201
(17 Vi spør) 0:00:00 0:00:00
23257 quiz 17 Quiz 201
(17 Quiz) 0:00:00 0:00:00
23252 nyhetsnotis 17 NYH NOTISER 201
(17 NYH NOTISER) 0:00:00 0:00:00
23192 nyhet 16 nyh karaktersak ekspert 201
(16 nyh karaktersak ekspert) 0:00:00 0:00:00
23214 nyhet 16 nyh Fem på hookern 400
23218 nyhet 16 notiser 201
(16 notiser) 0:00:00 0:00:00
23212 nyhet Hooke-nekt ved HiOA 201
(Hooke-nekt ved HiOA) 0:00:00 0:00:00
22587 nyhet ISS oppfølger 17 201
(ISS oppfølger 17) 0:00:00 0:00:00
23255 nyhet Svindyr faddermorro 201
(Svindyr faddermorro) 0:00:00 0:00:00
23174 magasin MAGCurlingbarn på Blindern-isen 201
(MAGCurlingbarn på Blindern-isen) 0:00:00 0:00:00
23225 magasin Tips til ferske studenter 201
(Tips til ferske studenter) 0:00:00 0:00:00
23223 leder 16 Leder  201
(16 Leder) 0:00:00 0:00:00
23261 leder 17 leder 201
(17 leder) 0:00:00 0:00:00
23199 kulturnotis Kulturnotiser 400
23245 kulturnotis 17 KUL NOTISER 201
(17 KUL NOTISER) 0:00:00 0:00:00
23268 kommentar 18 DEB nyhk

In [ ]:

Story._meta.get_field('modified').pre_save = pre_save

for story in Story.objects.filter(publication_status__lt=Story.STATUS_PUBLISHED):
    data = old_api.get(story.pk)
    created = parser(data.get('dato'))
    modified = parser(data.get('version_date'))
    story.created = created
    story.modified = modified
    story.publication_status = flags[data.get('flagg')]
    story.save(update_fields=['created', 'modified'] )
    print(story)
    print(created, modified, story.modified, sep='\n')

In [ ]:
import pytz
pytz.country_timezones('no')

In [ ]:
def get_story_type(prodsys_mappe):
    """ Find correct story type for the story based on legacy label. """
    try:
        story_type = StoryType.objects.filter(prodsys_mappe=prodsys_mappe)[0]
    except IndexError:
        generic_section, exists = Section.objects.get_or_create(
            title='New Section')
        story_type = StoryType.objects.create(
            prodsys_mappe=prodsys_mappe,
            name="New Story Type ({label})".format(label=prodsys_mappe),
            section=generic_section,
        )
    # finally:
    return story_type

In [ ]:
Story.objects.filter(publication_status__lte=1).delete()

In [ ]:
response = api.post(id=23215, data={'arbeidstittel': 'awesome', 'produsert': 0})
print(response.request.headers, response.request.body, sep='\n')
print(response.url, response.status_code, response.text)
testsak = api.get(23215)
print(testsak['arbeidstittel'], testsak['version_no'], testsak['produsert'])

In [ ]:
from django.contrib.auth.models import Permission
pp = Permission.objects.filter(codename__contains='_story')
for p in pp: print(p.codename)jjj

In [ ]:
from django.contrib.auth import authenticate
create_user = get_user_model().objects.create
get_user_model().objects.filter(username='api').delete()
api_user = get_user_model().objects.create_user(
    username='api',
    password='api',
    email='api@api.com',
)
print(api_user)
login = authenticate(username='api', password='api')
print(login)

In [ ]:
from django.contrib.auth import get_user_model

def api_user():
    User = get_user_model()
    try:
        user = User.objects.get(username='api')
    except User.DoesNotExist:
        user = User.objects.create_user(
            username='api',
            password='api',
            email='api@example.com',
        )
    permissions = list(Permission.objects.filter(codename__contains='_story'))
    user.user_permissions.add(*permissions)
    for p in permissions:
        print(p.codename)
    return user

api_user()

In [ ]:
saker = api.list()
til_web = list(filter(lambda sak: sak['produsert'] == '6', saker))
pprint(til_web[7])


In [ ]:
pprint(api.get(23194))

In [ ]:
def deserialize_story(story):
    output = [] 
    head_tags = [('tit', 'title'),('ing', 'lede'),('tema', 'theme_word'),('stikktit', 'kicker')]
    for tag, attr in head_tags:
        text = getattr(story, attr)
        if text:
            output.append(f'@{tag}:{text}')
    for bl in story.byline_set.select_related('contributor'):
        output.append(str(bl))
    output.append(story.bodytext_markup) 
    for aside in story.asides():
        output.append(aside.child.bodytext_markup)
    for pullquote in story.pullquotes():
        output.append(pullquote.child.bodytext_markup)
    return '\n'.join(output)
    
story = Story.objects.order_by('?').last()
txt1 = deserialize_story(story)
prodstory = api.get(story.prodsak_id)
txt2 = prodstory.get('tekst')
print(story.prodsak_id)
print(txt1)
print(txt2)

In [ ]:
story = Story.objects.get(id=61090)
story.images()

In [ ]:
imgstories = Story.objects.filter(pk__in=StoryImage.objects.all().values_list('parent_story', flat=True).distinct())
imgstories[0].pk
